In [0]:
x=1
y=4
x+y,x*y

Out[1]: (5, 4)

Sum of list

In [0]:
listn = [ 1,2,3,4,5]
sum(listn)

Out[2]: 15

Calculate Average

In [0]:
sum(listn)/len(listn)

Out[3]: 3.0

Dictionary

In [0]:
my_dict = {"name":"Alice", "age": 25}
print(my_dict["name"])
my_dict["age"] = 26
print(my_dict.keys())

Alice
dict_keys(['name', 'age'])


In [0]:
my_set = {1,2,3,4,5}
my_set.add(6)
print(my_set)
my_set.update([2,7,8]) # adding multiple values
my_set

{1, 2, 3, 4, 5, 6}
Out[5]: {1, 2, 3, 4, 5, 6, 7, 8}

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()

# Creating DataFrame from list of tuples
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+

3
+-------+---+-----+-------+
|summary| id| name|country|
+-------+---+-----+-------+
|  count|  3|    3|      3|
|   mean|2.0| null|   null|
| stddev|1.0| null|   null|
|    min|  1|Alice|  India|
|    max|  3|Carol|  India|
+-------+---+-----+-------+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A", 10, 200.0),
    ("2024-01-01", "South", "Product B", 5, 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East",  "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)
sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
sales_data_total = [(i[2],i[3]*i[4]) for i in sales_data]
columns = ["product","Total Revenue"]

sales_df3 = spark.createDataFrame(sales_data_total, columns)
sales_df3.show()

+---------+-------------+
|  product|Total Revenue|
+---------+-------------+
|Product A|       2000.0|
|Product B|       1500.0|
|Product A|       8000.0|
|Product B|       6000.0|
|Product C|       5625.0|
+---------+-------------+



In [0]:
from pyspark.sql.functions import sum
sales_df.groupBy('product').agg(sum('revenue')).show()

+---------+------------+
|  product|sum(revenue)|
+---------+------------+
|Product A|       600.0|
|Product B|       900.0|
|Product C|       375.0|
+---------+------------+



In [0]:
sales_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- revenue: double (nullable = true)



In [0]:
from pyspark.sql.functions import avg

sales_df.groupBy('product').agg(sum('revenue')/sum('quantity')).show()

+---------+------------------------------+
|  product|(sum(revenue) / sum(quantity))|
+---------+------------------------------+
|Product A|                          20.0|
|Product B|                          60.0|
|Product C|                          25.0|
+---------+------------------------------+



In [0]:
from pyspark.sql.functions import max
sales_df.groupBy('region').agg(sum('revenue')).sort('descending').show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2035596784820254>:2
      1 from pyspark.sql.functions import max
----> 2 sales_df.groupBy('region').agg(sum('revenue')).sort('descending').show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:2613, in DataFrame.sort(self, *cols, **kwargs)
   2522 def sort(
   2523     self, *cols: Union[str, Column, List[Union[str, Column]]], **kwargs: Any
   2524 ) -> "DataFrame":
   2525     """Returns a new :class:`DataFrame` sorted by the specified column(s).